In [23]:
import time

# getpass - to input our password without showing it in the notebook
from getpass import getpass

# Juicy stuff - these are the Classes we will use for interaction with a webpage:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from webdriver_manager.chrome import ChromeDriverManager
from selenium.common import NoSuchElementException, ElementClickInterceptedException

# libraries for interacting with the operating system (OS)
import pathlib
import os
from os.path import join

import pandas as pd
import random
import re
from bs4 import BeautifulSoup
import re

import warnings
warnings.filterwarnings('ignore')

import numpy as np


In [2]:
driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()))


KeyboardInterrupt: 

In [6]:
def perform_search_and_scroll(driver):
    try:
        # Find the element with the specified class name and click it
        button_search = driver.find_element(By.CLASS_NAME, 'zoeken-resultaten-lijst-meer')
        button_search.click()

        # Scroll to the end of the page
        driver.find_element(By.TAG_NAME, 'body').send_keys(Keys.END)

        # Optionally, you can add a delay to give the page time to load new content
        # import time
        # time.sleep(2)  # 2 seconds of delay (adjust as needed)
        
    except Exception as e:
        print(f"An error occurred: {str(e)}")

In [7]:
def extract_school_details (url):
    driver.get(url)
    
    #cookies
    import pickle
    cookies_dir = 'saved_cookies'
    lis_dir = os.listdir()

    if cookies_dir not in lis_dir:
        os.mkdir(cookies_dir)
    else:
        pass # os.removedirs(cookies_dir) --> to remove a directory

    save_location = cookies_dir + '/cookies.pkl'
    pickle.dump(driver.get_cookies() , open(save_location,"wb"))
    
    #button_search
    #button_search = driver.find_element(By.CLASS_NAME, 'zoeken-resultaten-lijst-meer')
    #button_search.click()
    
    #getting data
    html = driver.page_source
    soup = BeautifulSoup(html)
    soup.find_all(attrs={'class': re.compile(r'zoekresultaat-title')})
    
    #links
    school_links_dirty = soup.select('h3 a')
    school_links_clean = [item.get('href') for item in school_links_dirty]
    
    #names
    school_list_dirty = soup.find_all(attrs={'class': re.compile(r'zoekresultaat-title')})
    school_list_clean = [school.text.strip() for school in school_list_dirty]
    
    #adresses
    adress_list_dirty = soup.find_all(attrs={'class': re.compile(r'zoekresultaat-adres')})
    adress_list_clean = [job.text.strip() for job in adress_list_dirty]
    
    #details
    details_list_dirty = soup.find_all(attrs={'class': re.compile(r'zoekresultaat-details')})
    details_list_clean = [job.text.strip() for job in details_list_dirty]
    
    #number of students
    details_list_clean
    pattern = r'\d+'
    number_of_students = [re.search(pattern, item).group() if re.search(pattern, item) else '' for item in details_list_clean]
        
    return pd.DataFrame({'name':school_list_clean, 'address':adress_list_clean, 'link':school_links_clean, 'details':details_list_clean, 'n_students':number_of_students})

In [49]:
def create_database (csv):    
    mschool_links = pd.read_csv(csv)
    dfs_list = []
    for ind in mschool_links.index:
        df = (extract_school_details(mschool_links['Link'][ind]))
        df['neighborhood'] = mschool_links['Name'][ind]
        dfs_list.append(df)
    final_df = pd.concat(dfs_list, ignore_index=True)
    return final_df

In [50]:
final_mschool_df = create_database('places_middle_schools.csv')

In [51]:
final_mschool_df['type'] = 'Middle School'
final_mschool_df

,name,address,link,details,n_students,neighborhood,type
0,Yuverta vmbo Aalsmeer,Jac. P. Thijsselaan 18,/middelbare-scholen/aalsmeer/1713/yuverta-vmbo...,OverigeVoortgezet onderwijs535 leerlingen,535,Aalsmeer,Middle School
1,Yuverta mavo Aalsmeer,1e J.C. Mensinglaan 40,/middelbare-scholen/aalsmeer/1717/yuverta-mavo...,OverigeVoortgezet onderwijs491 leerlingen,491,Aalsmeer,Middle School
2,Chr Coll Schaersvoorde,Slingelaan 28,/middelbare-scholen/aalten/3289/chr-coll-schae...,Prot. christelijkVoortgezet onderwijs1019 leer...,1019,Aalten,Middle School
3,Chr Coll Schaersvoorde,Landbouwstraat 1,/middelbare-scholen/aalten/23137/chr-coll-scha...,Prot. christelijkVoortgezet onderwijs382 leerl...,382,Aalten,Middle School
4,Montessori College Aerdenhout,Boekenroodeweg 5,/middelbare-scholen/aerdenhout/2536/montessori...,SamenwerkingVoortgezet onderwijs427 leerlingen...,427,Aerdenhout,Middle School
...,...,...,...,...,...,...,...
1570,de Ambelt,Dokter Hengeveldweg 9,/middelbare-scholen/zwolle/26613/de-ambelt/,Algemeen bijzonderVoortgezet speciaal onderwij...,174,Zwolle,Middle School
1571,De Ambelt,Herfterlaan 39 F,/middelbare-scholen/zwolle/26615/de-ambelt/,Algemeen bijzonderVoortgezet speciaal onderwij...,209,Zwolle,Middle School
1572,Van der Capellen Sg,Lassuslaan 230,/middelbare-scholen/zwolle/22431/van-der-capel...,OpenbaarVoortgezet onderwijs1244 leerlingen,1244,Zwolle,Middle School
1573,De Twijn/Dr Itardschool,Dokter Hengeveldweg 2,/middelbare-scholen/zwolle/22935/de-twijn-dr-i...,OpenbaarVoortgezet speciaal onderwijs385 leerl...,385,Zwolle,Middle School


In [52]:
final_bschool_df = create_database('places_basic_schools.csv')

In [53]:
final_bschool_df['type'] = 'Basic School'
final_bschool_df

,name,address,link,details,n_students,neighborhood,type
0,Kindcentrum De Wegwijzer,Harm Tiesingstraat 74,/basisscholen/2e-exloërmond/25159/kindcentrum-...,Prot. christelijkBasisonderwijs84 leerlingen,84,2e Exloërmond,Basic School
1,Openbare Basisschool De Westhoek,Schoolstraat 27,/basisscholen/2e-exloërmond/7595/openbare-basi...,OpenbaarBasisonderwijs44 leerlingen,44,2e Exloërmond,Basic School
2,Basisschool De Tweeklank,Albardastraat 35,/basisscholen/aadorp/11785/basisschool-de-twee...,SamenwerkingBasisonderwijs116 leerlingenDalton,116,Aadorp,Basic School
3,JHR Willem Versluijsschool,Prelaatweg 22,/basisscholen/aagtekerke/25963/jhr-willem-vers...,Prot. christelijkBasisonderwijs191 leerlingen,191,Aagtekerke,Basic School
4,Christelijke Basisschool de Schutse,Paardelandsdrift 9,/basisscholen/aalden/9661/christelijke-basissc...,Prot. christelijkBasisonderwijs62 leerlingen,62,Aalden,Basic School
...,...,...,...,...,...,...,...
5619,De Ichthus,Zwarteweg 74,/basisscholen/zwolle/9451/de-ichthus/,Prot. christelijkBasisonderwijs317 leerlingenIPC,317,Zwolle,Basic School
5620,De Zuidster,Munterkamp 71,/basisscholen/zwolle/9452/de-zuidster/,Prot. christelijkBasisonderwijs87 leerlingen,87,Zwolle,Basic School
5621,PC Basisschool De Ark,Gedeputeerdenlaan 47,/basisscholen/zwolle/5569/pc-basisschool-de-ark/,Prot. christelijkBasisonderwijs225 leerlingen,225,Zwolle,Basic School
5622,Chr. Basisschool Het Mozaiek,Hortensiastraat 31,/basisscholen/zwolle/5570/chr-basisschool-het-...,Prot. christelijkBasisonderwijs170 leerlingen,170,Zwolle,Basic School


In [54]:
df_final = pd.concat([final_mschool_df,final_bschool_df], ignore_index=True)
df_final

,name,address,link,details,n_students,neighborhood,type
0,Yuverta vmbo Aalsmeer,Jac. P. Thijsselaan 18,/middelbare-scholen/aalsmeer/1713/yuverta-vmbo...,OverigeVoortgezet onderwijs535 leerlingen,535,Aalsmeer,Middle School
1,Yuverta mavo Aalsmeer,1e J.C. Mensinglaan 40,/middelbare-scholen/aalsmeer/1717/yuverta-mavo...,OverigeVoortgezet onderwijs491 leerlingen,491,Aalsmeer,Middle School
2,Chr Coll Schaersvoorde,Slingelaan 28,/middelbare-scholen/aalten/3289/chr-coll-schae...,Prot. christelijkVoortgezet onderwijs1019 leer...,1019,Aalten,Middle School
3,Chr Coll Schaersvoorde,Landbouwstraat 1,/middelbare-scholen/aalten/23137/chr-coll-scha...,Prot. christelijkVoortgezet onderwijs382 leerl...,382,Aalten,Middle School
4,Montessori College Aerdenhout,Boekenroodeweg 5,/middelbare-scholen/aerdenhout/2536/montessori...,SamenwerkingVoortgezet onderwijs427 leerlingen...,427,Aerdenhout,Middle School
...,...,...,...,...,...,...,...
7194,De Ichthus,Zwarteweg 74,/basisscholen/zwolle/9451/de-ichthus/,Prot. christelijkBasisonderwijs317 leerlingenIPC,317,Zwolle,Basic School
7195,De Zuidster,Munterkamp 71,/basisscholen/zwolle/9452/de-zuidster/,Prot. christelijkBasisonderwijs87 leerlingen,87,Zwolle,Basic School
7196,PC Basisschool De Ark,Gedeputeerdenlaan 47,/basisscholen/zwolle/5569/pc-basisschool-de-ark/,Prot. christelijkBasisonderwijs225 leerlingen,225,Zwolle,Basic School
7197,Chr. Basisschool Het Mozaiek,Hortensiastraat 31,/basisscholen/zwolle/5570/chr-basisschool-het-...,Prot. christelijkBasisonderwijs170 leerlingen,170,Zwolle,Basic School


In [55]:
df_final.to_csv('final_allschools_df.csv')